# Computer Vision, Image Classification - Covid CT Scans
Kyle Ziegler 4/2022

In [ ]:
import os
import datetime

import tensorflow as tf

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
"""Set Parameters"""

# Model
mode = 'transfer_learning'
num_classes = 4

# Data
data_path = 'gs://vertex-central-1f/covid_proj_tfrecords/train/TFRECORD*'
num_examples = 195000
loop_dataset = -1 # Use -1 to create infinite dataset
image_channels = 3
height, width = (300,300)

# Training
epochs = 100
batch_size = 256
steps_per_epoch = num_examples//batch_size

distribution_strategy = "mirrored" # Use mirrored or multi-worker mirrored

initial_learning_rate = 0.1
decay_steps = 100000
decay_rate = 0.96

# Logging
tb_resource_name = 'projects/156596422468/locations/us-central1/tensorboards/3919064061672685568'
log_dir = os.getcwd() + '/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_model_path = os.getcwd() + '/saved_models/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
histogram_freq = 1


### Distributed Training Setup
- Single machine with n number of GPUs, can be adjusted to anything in the [TF docs](https://www.tensorflow.org/guide/distributed_training)

In [ ]:


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
from keras.layers import BatchNormalization

if distribution_strategy == "mirrored":
    distribution_strategy = tf.distribute.MirroredStrategy()
else:
    distribution_strategy = tf.distribute.MultiWorkerMirroredStrategy()

with distribution_strategy.scope():
    def create_model():

        input_shape = (height, width, image_channels)
        input_layer = tf.keras.layers.Input(input_shape)

        # Base
        # base_layers = layers.experimental.preprocessing.Rescaling(1./255, name='bl_1')(input_layer)
        base_layers = layers.Conv2D(16, 3, padding='same', activation='relu', name='bl_2')(input_layer)
        base_layers = layers.MaxPooling2D(name='bl_3')(base_layers)
        base_layers = layers.Conv2D(32, 3, padding='same', activation='relu', name='bl_4')(base_layers)
        base_layers = layers.MaxPooling2D(name='bl_5')(base_layers)
        base_layers = layers.Conv2D(64, 3, padding='same', activation='relu', name='bl_6')(base_layers)
        base_layers = layers.MaxPooling2D(name='bl_7')(base_layers)
        base_layers = layers.Flatten(name='bl_8')(base_layers)

        # Classifier branch
        classifier_branch = layers.Dense(128, activation='relu', name='cl_1')(base_layers)
        classifier_branch = layers.Dense(num_classes, name='cl_head')(classifier_branch)
        # logisitic regression for each possible class

        # Localizer branch
        locator_branch = layers.Dense(128, activation='relu', name='bb_1')(base_layers)
        locator_branch = layers.Dense(64, activation='relu', name='bb_2')(locator_branch)
        locator_branch = layers.Dense(32, activation='relu', name='bb_3')(locator_branch)
        locator_branch = layers.Dense(4, activation='sigmoid', name='bb_head')(locator_branch)
        # output 4 floats, MSE loss metric

        model = tf.keras.Model(input_layer, outputs=[classifier_branch,locator_branch])

        return model
    
    def create_transfer_learning_model():
        
        base_model = tf.keras.applications.ResNet101V2(
            include_top=False,
            weights="imagenet",
            input_shape=(height, width, image_channels),
            pooling="avg",
            classifier_activation=None, # only used when you are including the top
        )
        base_model.trainable = False
        
        input_shape = (height, width, image_channels)
        input_layer = tf.keras.layers.Input(input_shape)
        
        base_layers = base_model(input_layer, training=False)
        
        # base_model.add(Flatten())
        # flatten = tf.keras.layers.Flatten()(base_model.layers[-1].output)
        
        # Classifier branch
        classifier_branch = layers.Dense(128, activation='relu', name='cl_1')(base_layers)
        classifier_branch = layers.Dense(num_classes, name='cl_head')(classifier_branch)
        # logisitic regression for each possible class

        # Localizer branch
        locator_branch = layers.Dense(128, activation='relu', name='bb_1')(base_layers)
        locator_branch = layers.Dense(64, activation='relu', name='bb_2')(locator_branch)
        locator_branch = layers.Dense(32, activation='relu', name='bb_3')(locator_branch)
        locator_branch = layers.Dense(4, activation='sigmoid', name='bb_head')(locator_branch)
        # output 4 floats, MSE loss metric

        model = tf.keras.Model(input_layer, outputs=[classifier_branch,locator_branch])
        return model

    if mode == 'transfer_learning':
        model = create_transfer_learning_model()
    else:
        model = create_model()
    
# Creates a png of the network
# tf.keras.utils.plot_model(model)


In [ ]:
model.summary()

In [ ]:
losses = { 
    "cl_head":tf.keras.losses.SparseCategoricalCrossentropy(),        
    "bb_head":tf.keras.losses.MSE
}

metrics = { 
    "cl_head": "accuracy",
    "bb_head": "mean_squared_error"
}

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(loss=losses, optimizer=opt, metrics=metrics)

In [ ]:
# Testing mode object
import model

# Allows model file to be adjusted, and will be recompiled with each 
# execution of this cell.
from importlib import reload 
reload(model)

from model import Model

model = Model((height, width), num_classes, image_channels)
model.create_model("transfer_learning","mirrored")
model.add_loss_function(initial_learning_rate, decay_steps, decay_rate)

m = model.get_model()

In [ ]:
def _parse_record(record):
    """Parse a single record, and create TF features"""

    feature_mapping = {
        'image': tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        'bounding_box': tf.io.FixedLenFeature([4], tf.float32),
        'target_class': tf.io.FixedLenFeature([], tf.int64),
    }
    
    file_rec = tf.io.parse_single_example(record, feature_mapping)
    
    image = file_rec["image"]
    image = tf.io.decode_png(image, channels=image_channels)
    image = tf.image.resize(image, (height, width))
    
    # image = tf.image.convert_image_dtype(image, tf.float32) # normalizes between [0,1]

    # Best practice to use per image standardization, results in higher performance as well
    image = tf.image.per_image_standardization(image) 
    print(image)
    
    # print(image)
    
    bounding_box = file_rec["bounding_box"]
    
    target_class = file_rec["target_class"]
    
    return image, (target_class, bounding_box)

def create_prefetch_dataset(file_path):
    """Create the tf.dataset object, used to feed into fit() function"""
    
    print('Creating Prefetch Dataset From', file_path)

    # list_files have shuffle True by default
    dataset = tf.data.Dataset.list_files(file_path)
    
    # A note on caching with cache(), you should only use this on small datasets 
    # that fit into memory, otherwise you'll crash your machine. Don't ask how I know.
    
    dataset = dataset.interleave(tf.data.TFRecordDataset, num_parallel_calls=tf.data.AUTOTUNE)\
    .map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)\
    .shuffle(3, reshuffle_each_iteration=False)\
    .batch(batch_size, drop_remainder=True)\
    .prefetch(tf.data.AUTOTUNE)\
    .repeat(loop_dataset)

    return dataset

dataset = create_prefetch_dataset(data_path)
print(dataset)

### Notes on performance
- When using a batch size of 256, main mem is around 15GB used, slightly increasing over each epoch.
- CPU is around 10 cores at 100%, fluctuating between 5-11 cores.
- GPU is around 75-95% utilization, only dropping for a second between epochs. GPU mem is around 15GB on a 16GB card.

In [ ]:
# Code for writing profile metrics, does not work in Vertex hosted Tensorboard 4/2022
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
#                                                       histogram_freq=1,
#                                                       profile_batch = '0,10',
#                                                       write_images=False
#                                                      )

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                      histogram_freq=histogram_freq,
                                                     update_freq='epoch')
    
history = model.fit(dataset, epochs=epochs, steps_per_epoch=steps_per_epoch, initial_epoch = 0, callbacks=[tensorboard_callback])

In [ ]:
%%bash
tb-gcp-uploader --tensorboard_resource_name \
  tb_resource_name \
  --logdir=log_dir \
  --experiment_name=model-fit --one_shot=True